In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv', sep=',')

In [ ]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df, title="Profiling Report")
# profile.to_notebook_iframe()

### Drop unnecessary columns:
- Name
- Ticket
- Cabin
- PassengerId

In [ ]:
df.drop(columns=['Name','Ticket','Cabin','PassengerId'],inplace=True)

In [ ]:
# df['Sex'].replace('female',0,inplace=True)
# df['Sex'].replace('male',1,inplace=True)

### Impute Missing Values in Age

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# Iterative Imputation
iter_imputer = IterativeImputer(random_state=42)
iter_imputed = iter_imputer.fit_transform(np.array(df['Age']).reshape(-1, 1))
df['Age'] = iter_imputed

In [ ]:
Y = df.loc[:, df.columns == 'Survived']
X = df.loc[:, df.columns != 'Survived']

In [ ]:
cols_to_encode = [
                    "Sex",
                    "Embarked"
                ]
X = pd.get_dummies(X, columns = cols_to_encode)
# X = X.drop(cols_to_encode, axis=1)
# X = pd.concat([X, dummies], axis=1)

In [ ]:
X.skew()

In [ ]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=302, output_distribution='normal')
X.loc[:,'Fare'] = qt.fit_transform(np.array(X['Fare']).reshape(-1, 1))

In [ ]:
from sklearn.preprocessing import StandardScaler
columns = X.columns

#data preprocessing
numeric_columns = ['Age', 'Fare']

# Create a StandardScaler instance
scaler = StandardScaler()
# features = X_train[col_names]
# scaler = StandardScaler().fit(features.values)
# features = scaler.transform(features.values)

# Fit the scaler on the data and transform the selected columns
X.loc[:,numeric_columns] = scaler.fit_transform(X[numeric_columns])
X.columns = columns

In [ ]:
X

In [ ]:
boxplot = X.boxplot() 
plt.xticks(rotation=90);

In [ ]:
X.hist(figsize=(30,15), bins = 20, layout=(3,5));

In [ ]:
print(Y.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=3, shuffle=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline

names = [
    "Logistic Regression",
    "SVC",
    "Random Forest",
    "Multi-Layer Perceptron",
    "GaussianNB",
    "XGB",
    "LightGBM",
    "CatBoost",
]

classifiers = [
    LogisticRegression(),
    SVC(),
    RandomForestClassifier(),
    MLPClassifier(),
    GaussianNB(),
    XGBClassifier(),
    LGBMClassifier(),
    CatBoostClassifier(verbose=0),
    #CatBoostClassifier(verbose=0, n_estimators=35, max_depth=6),
    
]

for name, clf in zip(names, classifiers):
        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train, y_train.values.ravel())
        score = clf.score(X_test, y_test.values.ravel())
        print(name, ": ", score)